In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, optimizers, datasets, Sequential

print('python version:', sys.version.split('\n')[0])
for m in [tf, np, pd, mpl, sns]:
    print(m.__name__, 'version:', m.__version__)

python version: 3.6.9 (default, Oct  8 2020, 12:12:24) 
tensorflow version: 2.3.1
numpy version: 1.18.5
pandas version: 1.1.3
matplotlib version: 3.3.2
seaborn version: 0.11.0


# Network Layers

In [2]:
x = tf.constant([2.,1.,0.1])
layer = layers.Softmax(axis=-1)
out = layer(x)
print(out)

tf.Tensor([0.6590012  0.24243298 0.09856589], shape=(3,), dtype=float32)


In [3]:
network = Sequential([
    layers.Dense(3, activation='relu'),
    layers.Dense(2, activation='relu')
])

x = tf.random.normal([4,3])
out = network(x)
print(out)

tf.Tensor(
[[0.        1.3407786]
 [0.        1.2194555]
 [0.        2.016048 ]
 [0.        1.1010207]], shape=(4, 2), dtype=float32)


In [4]:
layers_num = 2
network = Sequential([])
for _ in range(layers_num):
    network.add(layers.Dense(3, activation='relu'))
network.build(input_shape=(4,4))
network.summary()
for p in network.trainable_variables:
    print(p.name, p.shape)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (4, 3)                    15        
_________________________________________________________________
dense_3 (Dense)              (4, 3)                    12        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
dense_2/kernel:0 (4, 3)
dense_2/bias:0 (3,)
dense_3/kernel:0 (3, 3)
dense_3/bias:0 (3,)


# Compile, Fit and predict

In [5]:
train_db = tf.data.Dataset.from_tensor_slices((tf.random.normal([100, 28*28]), tf.random.normal([100,10]))) 
train_db = train_db.shuffle(60000).batch(20)
val_db = tf.data.Dataset.from_tensor_slices((tf.random.normal([20, 28*28]), tf.random.normal([20,10])))
val_db = val_db.batch(20)

sample = next(iter(train_db))
print(sample[0].shape, sample[1].shape)

network = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)
])
network.build(input_shape=(None, 28*28))
network.summary()
network.compile(
    optimizer=optimizers.Adam(lr=0.01),
    loss=losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
history = network.fit(train_db, epochs=5, validation_data=val_db, validation_freq=2)
history.history

(20, 784) (20, 10)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
5/5 [==============================] - 0s 3ms/step - loss: -2.2547 - accuracy: 0.0900

{'loss': [-2.254695177078247,
  -76.19269561767578,
  -986.3984985351562,
  -7703.36376953125,
  -34662.21484375],
 'accuracy': [0.09000000357627869,
  0.15000000596046448,
  0.14000000059604645,
  0.1599999964237213,
  0.2199999988079071],
 'val_loss': [-18.3756103515625, 130.1220245361328],
 'val_accuracy': [0.10000000149011612, 0.10000000149011612]}

In [6]:
x,y = next(iter(val_db))
print('predict x:', x.shape) 
out = network.predict(x) 
print(out.shape)

predict x: (20, 784)
(20, 10)


# Modle sqve and load

## tensor

In [7]:
network.save_weights('Model/weights.ckpt')
print('saved weights')
del network

saved weights


In [8]:
network = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)
])
network.build(input_shape=(None, 28*28))
network.compile(
    optimizer=optimizers.Adam(lr=0.01),
    loss=tf.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
network.load_weights('Model/weights.ckpt')
print('loaded weights!')

loaded weights!


## Network

In [9]:
network.save('Model/model.h5')
print('saved total model.')
del network

saved total model.


In [10]:
network = keras.models.load_model('Model/model.h5')

## Saved model

In [11]:
tf.saved_model.save(network, 'model-savedmodel')
print('saving savedmodel.')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model-savedmodel/assets
saving savedmodel.


In [12]:
network = tf.saved_model.load('model-savedmodel')